In [ ]:
#Load the necessary libraries

import pyarrow as pa
import pyarrow.parquet as pq
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Input, Lambda, BatchNormalization, Add, Dropout
from tensorflow.keras import layers, metrics, models, regularizers
from tensorflow.keras import models
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras import regularizers
import tensorflow.keras.backend as K
from tensorflow.keras import initializers
from tensorflow.keras.initializers import HeNormal
from tensorflow.keras.callbacks import TensorBoard
from tensorflow import keras
import joblib
import math
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.callbacks import LearningRateScheduler
from sklearn.model_selection import train_test_split


In [ ]:
#Load the datasets

def print_title(title):
    print(f'{50 * "="}')
    print(title)
    print(f'{50 * "="}')


print_title('Loading Data')
df = pq.read_table("C:\\Users\\Data\\crosssectionaldata\\X.parquet", use_threads=True).to_pandas()
X0_train = pq.read_table("C:\\Users\\Data\\X0_train.parquet", use_threads=True).to_pandas()
X0_test = pq.read_table("C:\\Users\\Data\\X0_test.parquet", use_threads=True).to_pandas()
X1_train = pq.read_table("C:\\Users\\Data\\X1_train.parquet", use_threads=True).to_pandas()
X1_test = pq.read_table("C:\\Users\\Data\\X1_test.parquet", use_threads=True).to_pandas()
print_title('Data loading complete')

In [ ]:
#Process and split the datasets for validation

X = df.iloc[:, 1:, ]
y1 = df.iloc[:, 0]

y = X.iloc[:, 0]
X = X.iloc[:, 1:, ]

X = X.values.astype(np.float32)
X0_train = X0_train.values.astype(np.float32)
X1_train = X1_train.values.astype(np.float32)
X0_test = X0_test.values.astype(np.float32)
X1_test = X1_test.values.astype(np.float32)

X_train, X_test, y_train, y_test, y1_train, y1_test = train_test_split(X, y, y1, test_size=0.2, random_state=0)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=0)
X0_train, X0_val = train_test_split(X0_train, test_size=0.2, random_state=0, shuffle=False)
X1_train, X1_val = train_test_split(X1_train, test_size=0.2, random_state=0, shuffle=False)


In [ ]:
#Scale the datasets

scaler = MinMaxScaler()

# Fit the scaler on the training data and transform the training set
X_train_scaled = scaler.fit_transform(X_train)

# Transform the validation and test sets using the same scaler
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# Similarly, fit and transform X0_train, X0_val, X0_test for the second dataset
X0_train_scaled = scaler.transform(X0_train)
X0_val_scaled = scaler.transform(X0_val)
X0_test_scaled = scaler.transform(X0_test)

# And fit and transform X1_train, X1_val, X1_test for the third dataset
X1_train_scaled = scaler.transform(X1_train)
X1_val_scaled = scaler.transform(X1_val)
X1_test_scaled = scaler.transform(X1_test)

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)


In [ ]:
input_shape = (332909,)
latent_dim = 25
batch_size = 54

In [ ]:
#Autoencoder Architecture

def residual_block(x, filters, kernel_size=3, stride=1, l1_reg=0.01, l2_reg=0.01, dropout_rate=0.4, initializer='he_normal'):
    shortcut = x
    x = layers.Dense(filters, activation="relu", 
                     kernel_regularizer=regularizers.l1_l2(l1=l1_reg, l2=l2_reg),
                     kernel_initializer=initializer)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(dropout_rate)(x)
    x = layers.Dense(filters, activation=None, 
                     kernel_regularizer=regularizers.l1_l2(l1=l1_reg, l2=l2_reg),
                     kernel_initializer=initializer)(x)
    x = layers.BatchNormalization()(x)
    
    # Adjust the shortcut if necessary
    if shortcut.shape[-1] != filters:
        shortcut = layers.Dense(filters, activation=None, 
                                kernel_regularizer=regularizers.l1_l2(l1=l1_reg, l2=l2_reg),
                                kernel_initializer=initializer)(shortcut)
        shortcut = layers.BatchNormalization()(shortcut)
    
    x = layers.add([x, shortcut])
    x = layers.ReLU()(x)
    return x


def build_encoder(input_shape, latent_dim, l1_reg=0.01, l2_reg=0.01, dropout_rate=0.4, initializer='he_normal'):
    encoder_inputs = layers.Input(shape=input_shape)
    x = layers.Dense(600, activation="relu", 
                     kernel_regularizer=regularizers.l1_l2(l1=l1_reg, l2=l2_reg),
                     kernel_initializer=initializer)(encoder_inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(dropout_rate)(x)
    
    # Add residual blocks
    x = residual_block(x, 600, l1_reg=l1_reg, l2_reg=l2_reg, initializer=initializer)
    x = residual_block(x, 400, l1_reg=l1_reg, l2_reg=l2_reg, dropout_rate=dropout_rate, initializer=initializer)
    x = residual_block(x, 200, l1_reg=l1_reg, l2_reg=l2_reg, initializer=initializer)
    x = residual_block(x, 100, l1_reg=l1_reg, l2_reg=l2_reg, dropout_rate=dropout_rate, initializer=initializer)
    x = residual_block(x, 50, l1_reg=l1_reg, l2_reg=l2_reg, dropout_rate=dropout_rate)
        
    z_mean = layers.Dense(latent_dim, name="z_mean", kernel_initializer=initializer)(x)
    z_log_var = layers.Dense(latent_dim, name="z_log_var", kernel_initializer=initializer)(x)
    z = Sampling()([z_mean, z_log_var])
    encoder = models.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")
    return encoder


encoder = build_encoder(input_shape, latent_dim, l1_reg=0.01, l2_reg=0.01, dropout_rate=0.4, initializer='he_normal')

def build_decoder(latent_dim, output_shape, l1_reg=0.01, l2_reg=0.01, dropout_rate=0.4, initializer='he_normal'):
    latent_inputs = layers.Input(shape=(latent_dim,))
    x = layers.Dense(50, activation="relu", 
                     kernel_regularizer=regularizers.l1_l2(l1=l1_reg, l2=l2_reg),
                     kernel_initializer=initializer)(latent_inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(dropout_rate)(x)
    
    # Add residual blocks
    
    x = residual_block(x, 50, l1_reg=l1_reg, l2_reg=l2_reg, initializer=initializer)
    x = residual_block(x, 100, l1_reg=l1_reg, l2_reg=l2_reg, dropout_rate=dropout_rate, initializer=initializer)
    x = residual_block(x, 200, l1_reg=l1_reg, l2_reg=l2_reg, initializer=initializer)
    x = residual_block(x, 400, l1_reg=l1_reg, l2_reg=l2_reg, dropout_rate=dropout_rate, initializer=initializer)
    x = residual_block(x, 600, l1_reg=l1_reg, l2_reg=l2_reg, initializer=initializer)
    
    decoder_outputs = layers.Dense(output_shape, activation="sigmoid", kernel_initializer=initializer)(x)  # Use linear activation for real-valued outputs
    decoder = models.Model(latent_inputs, decoder_outputs, name="decoder")
    return decoder


decoder = build_decoder(latent_dim, 332909, l1_reg=0.01, l2_reg=0.01, dropout_rate=0.4, initializer='he_normal')


In [ ]:
#Projector block where Latent space is reduced to 1D scalar value - DAI

class ScalarTransformation(layers.Layer):
    def __init__(self, **kwargs):
        super(ScalarTransformation, self).__init__(**kwargs)
        self.dense = layers.Dense(1, activation=None)  # Single scalar output

    def call(self, inputs):
        return self.dense(inputs)

In [ ]:
#Variational autoencoder class

class VAE(keras.Model):
    def __init__(self, encoder, decoder, scalar_transformation, **kwargs):
        super().__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.scalar_transformation = scalar_transformation
      

    def call(self, inputs):
        print(len(inputs))
        z_mean_cross, z_log_var_cross, z_cross = self.encoder(inputs[0])
        z_mean_present, z_log_var_present, z_present = self.encoder(inputs[1])
        z_mean_future, z_log_var_future, z_future = self.encoder(inputs[2])

        scalar_present = self.scalar_transformation(z_present)
        scalar_future = self.scalar_transformation(z_future)
        
        reconstruction = self.decoder(z_cross)

        return reconstruction, z_present, z_future, z_mean_cross, z_log_var_cross, scalar_present, scalar_future

In [ ]:
#Koopman operator class

class KoopmanOperator(tf.Module):
    def __init__(self, params):
        super().__init__()
        self.params = params
        self.latent_dim = params['latent_dim']  # Use latent_dim directly from params, no default
        
        # Create all omega networks once during initialization
        self.omega_nets = self.create_all_omega_nets()
        
        # Create transformation layer once during initialization
        self.transformation_layer = layers.Dense(1, activation=None)
    
    def form_complex_conjugate_block(self, omegas, delta_t):
        scale = tf.exp(omegas[:, 1] * delta_t)
        entry11 = tf.multiply(scale, tf.cos(omegas[:, 0] * delta_t))
        entry12 = tf.multiply(scale, tf.sin(omegas[:, 0] * delta_t))
        row1 = tf.stack([entry11, -entry12], axis=1)  # [None, 2]
        row2 = tf.stack([entry12, entry11], axis=1)  # [None, 2]
        result = tf.stack([row1, row2], axis=2)
        print("form_complex_conjugate_block - result shape:", result.shape)
        return result

    def varying_multiply(self, y, omegas, delta_t):
        num_real = self.params.get('num_real', 0)
        num_complex_pairs = self.params.get('num_complex_pairs', 0)
        complex_list = []
        real_list = []

        for j in range(num_complex_pairs):
            ind = 2 * j
            ystack = tf.stack([y[:, ind:ind + 2], y[:, ind:ind + 2]], axis=2)  # [None, 2, 2]
            L_stack = self.form_complex_conjugate_block(omegas[j], delta_t)
            elmtwise_prod = tf.multiply(ystack, L_stack)
            complex_list.append(tf.reduce_sum(elmtwise_prod, 1))

        if len(complex_list) > 0:
            complex_part = tf.concat(complex_list, axis=1)
            print("varying_multiply - complex_part shape:", complex_part.shape)

        for j in range(num_real):
            ind = 2 * num_complex_pairs + j
            temp = y[:, ind]
            real_list.append(tf.multiply(temp[:, tf.newaxis], tf.exp(omegas[num_complex_pairs + j] * delta_t)))

        if len(real_list) > 0:
            real_part = tf.concat(real_list, axis=1)
            print("varying_multiply - real_part shape:", real_part.shape)

        # Ensure the final result has the correct shape
        if len(complex_list) > 0 and len(real_list) > 0:
            result = tf.concat([complex_part, real_part], axis=1)
            result = result[:, :self.latent_dim]  # Trim to latent_dim
            print("varying_multiply - result shape (complex + real):", result.shape)
            return result
        elif len(complex_list) > 0:
            return complex_part
        else:
            return real_part
        
    def create_all_omega_nets(self):
        omega_nets = []
        for j in range(self.params['num_complex_pairs']):
            temp_name = f'OC{j + 1}'
            omega_net = self.create_one_omega_net(temp_name)  # Create model
            omega_nets.append(omega_net)
    
        for j in range(self.params['num_real']):
            temp_name = f'OR{j + 1}'
            omega_net = self.create_one_omega_net(temp_name)  # Create model
            omega_nets.append(omega_net)
    
        print("create_all_omega_nets - number of omega_nets:", len(omega_nets))
        return omega_nets

    def create_one_omega_net(self, name_prefix):
        latent_inputs = layers.Input(shape=(self.latent_dim,))
        
        x = layers.Dense(8, activation="relu", name=f'{name_prefix}_dense1')(latent_inputs)
        x = layers.BatchNormalization(name=f'{name_prefix}_batchnorm1')(x)
        x = layers.Dropout(0.4, name=f'{name_prefix}_dropout1')(x)       
        
        x = residual_block(x, 8, l1_reg=0.01, l2_reg=0.01)
        x = residual_block(x, 4, l1_reg=0.01, l2_reg=0.01, dropout_rate=0.4)
        x = residual_block(x, 2, l1_reg=0.01, l2_reg=0.01)
        
        omega_params = layers.Dense(self.latent_dim, name=f'{name_prefix}_output')(x)
        omegas = tf.keras.Model(latent_inputs, omega_params, name=name_prefix)
        
        return omegas

    def apply_omega_nets(self, ycoords):
        omegas = []
        for j in range(self.params['num_complex_pairs']):
            ind = 2 * j
            pair_of_columns = ycoords[:, ind:ind + 2]
            radius_of_pair = tf.reduce_sum(tf.square(pair_of_columns), axis=1, keepdims=True)
            radius_of_pair = tf.tile(radius_of_pair, [1, self.latent_dim])
            omega_output = self.omega_nets[j](radius_of_pair)
            print(f"apply_omega_nets - omega_net {j} output shape:", omega_output.shape)
            omegas.append(omega_output)
    
        for j in range(self.params['num_real']):
            ind = 2 * self.params['num_complex_pairs'] + j
            one_column = ycoords[:, ind]
            one_column = tf.tile(one_column[:, tf.newaxis], [1, self.latent_dim])
            omega_output = self.omega_nets[self.params['num_complex_pairs'] + j](one_column)
            print(f"apply_omega_nets - omega_net {self.params['num_complex_pairs'] + j} output shape:", omega_output.shape)
            omegas.append(omega_output)
    
        return omegas

    def compute_future_state(self, current_state, delta_t):
        """
        Compute future state based on current state and varying delta_t.
        """
        ycoords = current_state
        omegas = self.apply_omega_nets(ycoords)
        
        # Adjust varying time steps (delta_t)
        future_state = self.varying_multiply(current_state, omegas, delta_t)
        
        # Apply transformation to future state
        trans_future_state = self.transformation_layer(future_state)
        
        return future_state, trans_future_state



# Example parameters for model creation
params = {
    'input_shape': (332909,),  # Example input shape
    'latent_dim': 25,          # Latent space dimension
    'l1_reg': 0.01,            # L1 regularization strength
    'l2_reg': 0.01,            # L2 regularization strength
    'dropout_rate': 0.4,       # Dropout rate
    'delta_t': 3,              # Time step size
    'num_real': 15,            # Number of real eigenvalues
    'num_complex_pairs': 5,   # Number of complex conjugate eigenvalue pairs
    'output_shape': 332909,    # Output shape
}

In [ ]:
#Defining the KoopmanModel class

class KoopmanModel(tf.keras.Model):
    def __init__(self, koopman_operator):
        super(KoopmanModel, self).__init__()
        self.koopman_operator = koopman_operator

    def call(self, input_present, num_future=1, time_intervals=None):
        future_states = []
        trans_future_states = []
        current_state = input_present
        
        # If no custom time intervals are provided, use a fixed delta_t
        if time_intervals is None:
            time_intervals = [self.koopman_operator.params['delta_t']] * num_future
        
        for i in range(num_future):
            delta_t = time_intervals[i]  # Use the appropriate delta_t for each step
            g_next_state, trans_future_state = self.koopman_operator.compute_future_state(current_state, delta_t)
            
            future_states.append(g_next_state)
            trans_future_states.append(trans_future_state)
            
            current_state = g_next_state  # Update current state to the newly predicted state
        
        return future_states, trans_future_states


In [ ]:
#The entire architecture with custom loss functions and gradient initiation

class MyModel(tf.keras.Model):
    def __init__(self, vae, koopman, loss_weights=None, **kwargs):
        super().__init__(**kwargs)
        self.vae = vae
        self.koopman = koopman        
        self.total_loss_tracker = metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = metrics.Mean(name="reconstruction_loss")
        self.kl_loss_tracker = metrics.Mean(name="kl_loss")
        self.linear_dynamics_loss_tracker = metrics.Mean(name="linear_dynamics_loss")
        self.future_state_loss_tracker = metrics.Mean(name="future_state_loss")
        self.aux_loss_tracker = metrics.Mean(name="aux_loss")
        self.l_inf_loss_tracker = metrics.Mean(name="l_inf_loss")

        if loss_weights is None:
            loss_weights = {
                "reconstruction_loss": 10.0,
                "kl_loss": 1.0,
                "linear_dynamics_loss": 100.0,
                "future_state_loss": 100.0,
                "l_inf_loss": 1.0
            }
        self.loss_weights = loss_weights

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
            self.linear_dynamics_loss_tracker,
            self.future_state_loss_tracker,
            self.aux_loss_tracker,
            self.l_inf_loss_tracker
        ]
    
    def call(self, inputs, num_future=1, time_intervals=None):
        """
        Call method adjusted to handle multiple future time intervals (3 years and 10 years).
        """
        reconstruction, z_present, z_future, z_mean_cross, z_log_var_cross, scalar_present, scalar_future = self.vae(inputs)
        
        future_states = []
        trans_future_states = []
        
        current_state = z_present
        
        # Adjust for varying time intervals
        if time_intervals is None:
            time_intervals = [self.koopman.koopman_operator.params['delta_t']] * num_future
        
        for i in range(num_future):
            # Use the respective time interval for each future step
            delta_t = time_intervals[i]
            g_next_state, trans_future_state = self.koopman.koopman_operator.compute_future_state(current_state, delta_t)
            future_states.append(g_next_state)
            trans_future_states.append(trans_future_state)
            
            # Update current state to the newly predicted state
            current_state = g_next_state

        aux_reconstructed = [self.vae.decoder(g_next) for g_next in future_states]
        
        # Return the states
        return reconstruction, z_present, z_future, z_mean_cross, z_log_var_cross, scalar_present, scalar_future, future_states, trans_future_states, aux_reconstructed


    def compute_losses(self, inputs, reconstruction, z_present, z_future, z_mean_cross, z_log_var_cross, scalar_present, scalar_future, k_trans, k_untrans, aux_reconstructed, num_future=1):
        input_data_cross, input_data_present, input_data_future = inputs
    
        # Reconstruction loss (L_recon)
        reconstruction_loss = tf.reduce_mean(tf.reduce_sum(tf.square(input_data_cross - reconstruction), axis=-1))
        
        # KL divergence loss (remains as is)
        kl_loss = -0.5 * tf.reduce_mean(1 + z_log_var_cross - tf.square(z_mean_cross) - tf.exp(z_log_var_cross))
    
        # Future state prediction loss (L_pred)
        linear_dynamics_loss = 0.0
        for i in range(num_future):
            linear_dynamics_loss += tf.reduce_mean(tf.reduce_sum(tf.square(k_untrans[i] - z_future[i]), axis=-1))


        future_state_loss = 0.0
        for i in range(num_future):
            future_state_loss += tf.reduce_mean(tf.reduce_sum(tf.square(k_trans[i] - scalar_future[i]), axis=-1))


        aux_loss = tf.reduce_mean(tf.reduce_sum(tf.square(aux_reconstructed[-1] - input_data_future), axis=-1))
    
        # Infinity Norm Loss (L_inf)
        l_inf_loss = tf.reduce_max(tf.abs(input_data_cross - reconstruction)) + tf.reduce_max(tf.abs(input_data_future - aux_reconstructed[-1]))
    
        # Apply loss weights
        total_loss = (
            self.loss_weights["reconstruction_loss"] * (reconstruction_loss + aux_loss) +
            self.loss_weights["linear_dynamics_loss"] * linear_dynamics_loss +  
            self.loss_weights["kl_loss"] * kl_loss +
            self.loss_weights["future_state_loss"] * future_state_loss +
            self.loss_weights["l_inf_loss"] * l_inf_loss
        )
        
        return total_loss, reconstruction_loss, kl_loss, linear_dynamics_loss, future_state_loss, aux_loss, l_inf_loss

    def train_step(self, data):
        data_unpacked = data[0]
        input_data_cross, input_data_present, input_data_future = data_unpacked
        
        # Define time intervals: 3-year and 10-year prediction
        time_intervals = [3]  # Use [3] for single-step prediction or modify as needed
        
        with tf.GradientTape() as tape:
            reconstruction, z_present, z_future, z_mean_cross, z_log_var_cross, scalar_present, scalar_future, k_untrans, k_trans, aux_reconstructed = self(
                data_unpacked, num_future=len(time_intervals), time_intervals=time_intervals, training=True
            )
        
            # Compute losses
            total_loss, reconstruction_loss, kl_loss, linear_dynamics_loss, future_state_loss, aux_loss, l_inf_loss = self.compute_losses(
                (input_data_cross, input_data_present, input_data_future),
                reconstruction, z_present, z_future, z_mean_cross, z_log_var_cross, scalar_present, scalar_future, k_trans, k_untrans, aux_reconstructed, num_future=len(time_intervals)
            )
        
        # Compute gradients
        gradients = tape.gradient(total_loss, self.trainable_variables)
        
        # Apply gradients
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))
        
        # Update metrics
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        self.linear_dynamics_loss_tracker.update_state(linear_dynamics_loss)
        self.future_state_loss_tracker.update_state(future_state_loss)
        self.aux_loss_tracker.update_state(aux_loss)
        self.l_inf_loss_tracker.update_state(l_inf_loss)
        
        return {m.name: m.result() for m in self.metrics}
    
    def test_step(self, data):
        data_unpacked = data[0]
        input_data_cross, input_data_present, input_data_future = data_unpacked
        
        # Define time intervals for testing: e.g., 3-year and 10-year prediction
        time_intervals = [3]
        
        reconstruction, z_present, z_future, z_mean_cross, z_log_var_cross, scalar_present, scalar_future, k_trans, k_untrans, aux_reconstructed = self(
            data_unpacked, num_future=len(time_intervals), time_intervals=time_intervals, training=False
        )
        
        # Compute losses
        total_loss, reconstruction_loss, kl_loss, linear_dynamics_loss, future_state_loss, aux_loss, l_inf_loss = self.compute_losses(
            (input_data_cross, input_data_present, input_data_future),
            reconstruction, z_present, z_future, z_mean_cross, z_log_var_cross, scalar_present, scalar_future, k_trans, k_untrans, aux_reconstructed, num_future=len(time_intervals)
        )
        
        # Update metrics
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        self.linear_dynamics_loss_tracker.update_state(linear_dynamics_loss)
        self.future_state_loss_tracker.update_state(future_state_loss)
        self.aux_loss_tracker.update_state(aux_loss)
        self.l_inf_loss_tracker.update_state(l_inf_loss)
        
        return {m.name: m.result() for m in self.metrics}


In [ ]:
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return "{}:{:>02}:{:>05.2f}".format(h, m, s)

tf.get_logger().setLevel('ERROR')

In [ ]:
def lr_scheduler(epoch, lr):
    if epoch % 250 == 0 and epoch != 0:
        return lr * 0.1  # reduce learning rate by a factor of 10
    else:
        return lr

# Create a learning rate scheduler callback
lr_scheduler_callback = LearningRateScheduler(lr_scheduler)

model.save_weights(filepath.format(epoch=0))

In [ ]:
if __name__ == '__main__':
    import time

    start = time.time()

    scalar_transformation = ScalarTransformation()    
    vae = VAE(encoder, decoder, scalar_transformation)
    koopman_operator = KoopmanOperator(params)
    koopman = KoopmanModel(koopman_operator)
    
    model = MyModel(vae, koopman)
    checkpoint_path = 'C:\\Users\\Best model\\saved-model-{epoch:02d}DAF.ckpt'
    checkpoint = tf.train.Checkpoint(model=model)

    model.compile(optimizer=keras.optimizers.Adam(learning_rate = 0.0001, clipvalue=1.0, clipnorm=1.0))
    

In [ ]:
#Begins the training

with tf.device('/GPU:0'):
    hist = model.fit(
        train_loader,
        epochs=1000,
        validation_data=val_loader,
        validation_freq=1,
        callbacks=[checkpoint]
    )

    elapsed = time.time() - start
    print(f'Training time: {hms_string(elapsed)}')
#     print(hist.history)